In [1]:
from trainlib.FileCollection import FileCollection
from trainlib.Preprocessor import Preprocessor
from trainlib.PCAWhiteningPreprocessor import PCAWhiteningPreprocessor
from trainlib.ConfigFileUtils import ConfigFileUtils
import trainlib.cuts as cuts
import pandas as pd
import numpy as np
import re

Welcome to JupyROOT 6.10/09


In [2]:
candidate_branches = ["PFMET", "nCleanedJetsPt30", "nCleanedJetsPt30BTagged_bTagSF", "nExtraLep", "ZZMass", "nExtraZ", "Z1Mass", "Z2Mass", "Z1Pt", "Z2Pt", "ZZMassErr", "ZZPt", "ZZEta", "ZZPhi", "Z1Flav", "Z2Flav"]
MELA_branches = ["D_VBF2j_ggH_ME", "D_VBF1j_ggH_ME", "D_WHh_ggH_ME", "D_ZHh_ggH_ME", "D_WHh_ZHh_ME", "D_VBF2j_WHh_ME", "D_VBF2j_ZHh_ME"]
allbranches = ["JetPt", "JetEta", "JetPhi", "LepPt", "LepEta", "LepPhi", "ExtraLepPt", "ExtraLepEta", "ExtraLepPhi"] + candidate_branches + MELA_branches + ["LHEAssociatedParticleId", "GenAssocLep1Id", "GenAssocLep2Id", "training_weight"]

In [9]:
class FlexiblePCAWhiteningPreprocessor(Preprocessor):
    def __init__(self, name, nonperiodic_columns, periodic_columns, cuts, cuts_s = None):
        self.name = name
        self.nonperiodic_columns = nonperiodic_columns
        self.periodic_columns = periodic_columns
        self.cuts = cuts
        self.cuts_s = cuts_s
        self.base_preprocessor = None
        self.processed_columns = None
        self.last_indices = None

    @classmethod
    def from_config(cls, config_section):
        preprocessor_name = re.sub('[<>]', '', config_section.name)
        nonperiodic_columns = ConfigFileUtils.parse_list(config_section['nonperiodic_columns'], lambda x: x)
        periodic_columns = ConfigFileUtils.parse_list(config_section['periodic_columns'], lambda x: x)
        preprocessor_cuts, cuts_s = ConfigFileUtils.parse_lambda_s(config_section['preprocessor_cuts'])
        
        obj = cls(name = preprocessor_name, nonperiodic_columns = nonperiodic_columns, periodic_columns = periodic_columns, cuts = preprocessor_cuts, cuts_s = cuts_s)
        
        return obj

    def to_config(self, confhandler):
        section_name = '<' + self.name
        confhandler.new_section(section_name)
        
        confhandler.set_field(section_name, 'preprocessor_type', 'FlexiblePCAWhiteningPreprocessor')
        confhandler.set_field(section_name, 'nonperiodic_columns', ConfigFileUtils.serialize_list(self.nonperiodic_columns, lambda x: x))
        confhandler.set_field(section_name, 'periodic_columns', ConfigFileUtils.serialize_list(self.periodic_columns, lambda x: x))
        confhandler.set_field(section_name, 'preprocessor_cuts', ConfigFileUtils.serialize_lambda_s(self.cuts, self.cuts_s))
    
    def setup_generator(self, datagen, len_setupdata):
        self.len_setupdata = len_setupdata
        
        # draw some data from the generator
        extracted_data = []
        extracted_rows = 0
        
        for data in datagen:
            extracted_data.append(data)
            extracted_row += len(data)
            
            if extracted_rows > self.len_setupdata:
                break
                
        input_data = pd.concat(extracted_data)
        input_data = input_data.reset_index(drop = True)
        
        self.setup(input_data)
    
    def setup(self, data):
        # first perform the row selection
        cut_data = self._row_cut(data)
        
        # then prepare the data and the base preprocessor
        prepared_data = self.prepare_data(cut_data)
        self.processed_columns = prepared_data.columns
        
        self.base_preprocessor = PCAWhiteningPreprocessor(self.name, self.processed_columns, cuts.no_cut, None)
        self.base_preprocessor.setup(prepared_data)
        
    def get_last_indices(self):
        return self.last_indices
    
    def process(self, data):
        cut_data = self._row_cut(data)        
        self.last_indices = cut_data.index
        
        prepared_data = self.prepare_data(cut_data)
                
        return self.base_preprocessor.process(prepared_data)
    
    def save(self, folder, filename):
        self.base_preprocessor.save(folder, filename)
        
    def load(self, folder, filename):
        self.base_preprocessor.load(folder, filename)
    
    # puts the data in a format such that PCAWhiteningPreprocessor can act on them
    def prepare_data(self, data):
        prepared_data = pd.DataFrame()

        for column in self.nonperiodic_columns:
            cf = self.extract_column(df, column)
            prepared_data = pd.concat([prepared_data, cf], axis = 1)

        for column in self.periodic_columns:
            cf = self.extract_column(df, column)
            cf = self.encode_periodic_column(cf)
            prepared_data = pd.concat([prepared_data, cf], axis = 1)

        return prepared_data

    def extract_order(self, df, sorted_column, columns, order):
        def get_index(row, order, sorted_column):
            sorted_column = row[sorted_column]
            if order >= len(sorted_column):
                return -1
            else:
                return np.flipud(np.argsort(sorted_column))[order]

        index_column = pd.DataFrame(df.transform(lambda row: get_index(row, order, sorted_column), axis = 1, raw = True))
        index_column.columns = ["index"]
        df_temp = pd.concat([index_column, df], axis = 1)

        def get_element(row, column_name):
            if row["index"] == -1:
                return 0
            else:
                return row[column_name][row["index"]]

        extracted_cols = pd.DataFrame()
        for column in columns:
            extracted_col = pd.DataFrame(df_temp.transform(lambda row: get_element(row, column), axis = 1, raw = True))
            extracted_col.columns = [column + "_" + str(order)]
            extracted_cols = pd.concat([extracted_cols, extracted_col], axis = 1)

        return extracted_cols
    
    def extract_order_filtered(self, df, sorted_column, columns, order):
        extracted_raw = self.extract_order(df, sorted_column, columns, order)

        if any("Jet" in col for col in extracted_raw.columns):
            # apply the jet-pt cut
            mask_column = self.extract_order(df, "JetPt", ["JetPt"], order)
            mask = mask_column < 30.0
            extracted_raw[mask.as_matrix()] = 0.0

        return extracted_raw
    
    def extract_column(self, df, colstring):
        if '[' in colstring and ']' in colstring:
            col, sorted_col_order, _ = re.split('[\[\]]', colstring)
            sorted_col, order = re.split('\|', sorted_col_order)
            cf = self.extract_order_filtered(df, sorted_col, [col], int(order))
        else:
            cf = df[[colstring]]
        return cf
    
    def encode_periodic_column(self, df):
        of = pd.DataFrame()
        sincol = df.apply(np.sin, raw = True)
        sincol.columns = [col + "_sin" for col in df.columns]
        coscol = df.apply(np.cos, raw = True)
        coscol.columns = [col + "_cos" for col in df.columns]
        of = pd.concat([sincol, coscol], axis = 1)

        return of
    
    def _row_cut(self, data):
        output_data = data.loc[data.apply(self.cuts, axis = 1)]
        return output_data

In [10]:
MC_path = "/data_CMS/cms/wind/CJLST_NTuples/"

In [17]:
coll = FileCollection({MC_path + "VBFH125/ZZ4lAnalysis.root": cuts.mZZ_cut}, 0.0, 0.5)
df = coll.get_data(allbranches, 200, 400)

skimming /data_CMS/cms/wind/CJLST_NTuples/VBFH125/ZZ4lAnalysis.root
collection set up: 1 files, 62320 entries in total, 31160 of which will be used


In [18]:
nonperiodic_columns = ["ZZMass","JetEta[JetPt|0]", "JetPt[JetPt|0]"]
periodic_columns = ["JetPhi[JetPt|0]", "ZZPhi"]

In [19]:
pre = FlexiblePCAWhiteningPreprocessor("test", nonperiodic_columns, periodic_columns, cuts.no_cut)

In [20]:
pre.setup(df)

PCA setup for 'test': Index([u'ZZMass', u'JetEta_0', u'JetPt_0', u'JetPhi_0_sin', u'JetPhi_0_cos',
       u'ZZPhi_sin', u'ZZPhi_cos'],
      dtype='object')
PCAWhiteningPreprocessor for stream 'test'
setting up PCA whitening on 182 events
182 remaining after the cuts


In [21]:
test = pre.process(df)["test"]

In [22]:
pre.prepare_data(df)

,ZZMass,JetEta_0,JetPt_0,JetPhi_0_sin,JetPhi_0_cos,ZZPhi_sin,ZZPhi_cos
0,122.344070,3.303871,57.361485,-0.983930,-0.178554,0.717633,0.696421
1,123.417145,-4.101622,32.574715,-0.669016,-0.743248,-0.744628,0.667480
2,122.310593,-1.517997,50.751476,-0.607315,0.794461,0.333242,-0.942841
3,123.935692,-3.036364,113.747162,0.967589,-0.252532,-0.890965,-0.454072
4,123.242645,0.000000,0.000000,0.000000,1.000000,0.026559,-0.999647
5,124.532410,-2.495588,77.237236,0.083768,0.996485,-0.331367,-0.943502
6,123.790184,0.000000,0.000000,0.000000,1.000000,0.745815,-0.666153
7,125.737633,2.839484,163.723724,-0.397668,-0.917529,0.264002,0.964522
8,124.864723,-1.391279,238.874283,-0.826765,-0.562548,-0.982155,0.188073
9,122.803452,3.904966,74.805603,-0.965685,0.259718,0.992582,-0.121578


In [25]:
test

array([[-0.3638802 , -0.91109655, -1.74968031, ..., -1.42941275,
         0.44151793,  0.76544905],
       [-0.71783617,  1.63937521,  0.71406488, ...,  0.64675154,
         0.10465547,  2.64755519],
       [-0.45828798,  0.90849814, -0.47082078, ..., -0.84029849,
        -0.75687354,  0.98584085],
       ..., 
       [-0.57117187,  2.18712712, -1.56872704, ...,  0.62357936,
        -0.42115719,  0.10642452],
       [ 0.27881501,  1.24626137, -0.8303008 , ..., -1.12977196,
        -1.21918221, -0.04276018],
       [ 0.36519844,  0.15433412, -1.38707013, ...,  0.27247079,
        -0.43107047, -1.25666884]])

In [24]:
df[["ZZMass"]]

,ZZMass
0,122.344070
1,123.417145
2,122.310593
3,123.935692
4,123.242645
5,124.532410
6,123.790184
7,125.737633
8,124.864723
9,122.803452


In [29]:
for x in ["a", "b",
          "c"]:
    print x

a
b
c


In [41]:
test = ["a", "b", "c", ""]

In [42]:
len([name for name in test if not name])

1

In [43]:
filter(None, test)

['a', 'b', 'c']